In [1]:
import numpy as np

In [11]:
y_hat = (np.random.rand(10, 5) > 0.2).astype(np.float32)
y_hat

array([[1., 0., 1., 0., 0.],
       [1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 0., 1., 0., 1.],
       [0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1.]], dtype=float32)

In [13]:
m = [1, 2, 1, 1, 3]
np.diag(m)

array([[1, 0, 0, 0, 0],
       [0, 2, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 3]])

In [16]:
np.outer(m, m)

array([[1, 2, 1, 1, 3],
       [2, 4, 2, 2, 6],
       [1, 2, 1, 1, 3],
       [1, 2, 1, 1, 3],
       [3, 6, 3, 3, 9]])

In [15]:
np.diag(m) - np.outer(m, m)

array([[ 0, -2, -1, -1, -3],
       [-2, -2, -2, -2, -6],
       [-1, -2,  0, -1, -3],
       [-1, -2, -1,  0, -3],
       [-3, -6, -3, -3, -6]])

In [50]:
import numpy as np

def generate_prob_matrix(rows, cols):
    random_matrix = np.random.rand(rows, cols)
    row_sums = random_matrix.sum(axis=1, keepdims=True)
    probability_matrix = random_matrix / row_sums
    return probability_matrix

# Example usage:
matrix = generate_prob_matrix(3, 4)
print("Generated Matrix:")
print(matrix)
print("\nRow Sums Check:")
print(matrix.sum(axis=1))


Generated Matrix:
[[0.19331988 0.0457593  0.37342652 0.38749429]
 [0.03424102 0.1335862  0.31643557 0.51573721]
 [0.29352668 0.2982287  0.15553784 0.25270678]]

Row Sums Check:
[1. 1. 1.]


In [51]:
matrix.shape

(3, 4)

In [49]:
(((matrix.reshape(3, 4, 1) * np.eye(4)) - (matrix.reshape(3, 4, 1) @ matrix.reshape(3, 1, 4))) @ matrix.reshape(3, 4, 1)).squeeze()

array([[ 0.02163613, -0.00869798,  0.0009184 , -0.01385654],
       [-0.02346224,  0.05889068, -0.01225177, -0.02317667],
       [ 0.00285497,  0.04368378, -0.023314  , -0.02322475]])

In [70]:
import numpy as np

def softmax(X):
    """
    Computes the softmax of each row of the input X.
    Incorporates numerical stability by subtracting the max logit.
    """
    # X shape: (M, C)
    shift_x = X - np.max(X, axis=1, keepdims=True)
    exps = np.exp(shift_x)
    return exps / np.sum(exps, axis=1, keepdims=True)

def cross_entropy_loss(P, Y):
    """
    Computes the mean cross-entropy loss over the batch.
    L = -1/M * sum(sum(Y * log(P)))
    """
    M = Y.shape[0]
    # Small epsilon to prevent log(0)
    epsilon = 1e-12
    log_p = np.log(P + epsilon)
    loss = -np.sum(Y * log_p) / M
    return loss

def compute_gradients(P, Y):
    """
    Computes the gradient of the loss w.r.t the logits X.
    dL/dX = 1/M * (P - Y)
    """
    M = Y.shape[0]
    return (P - Y) / M

# --- Execution & Synthetic Data Generation ---

# 1. Setup Hyperparameters
M, C = 10, 4
np.random.seed(42)

# 2. Generate Synthetic Data
# Logits (X) from a normal distribution
X = np.random.randn(M, C)

# One-hot encoded labels (Y)
target_classes = np.random.randint(0, C, M)
Y = np.eye(C)[target_classes]

# 3. Forward Pass
P = softmax(X)
loss = cross_entropy_loss(P, Y)

# 4. Backward Pass
grad_X = compute_gradients(P, Y)

# --- Output Results ---
print(f"--- Forward Pass ---")
print(f"Loss: {loss:.6f}")
print(f"\n--- Probabilities (P) Matrix (Shape {P.shape}) ---")
print(P[:3])  # Showing first 3 samples for brevity

print(f"\n--- Gradient (dL/dX) Matrix (Shape {grad_X.shape}) ---")
print(grad_X[:3]) # Showing first 3 samples

--- Forward Pass ---
Loss: 1.483649

--- Probabilities (P) Matrix (Shape (10, 4)) ---
[[0.18235934 0.09664077 0.21207787 0.50892203]
 [0.09213474 0.09213625 0.56488278 0.25084624]
 [0.1735804  0.47755308 0.17463491 0.17423161]]

--- Gradient (dL/dX) Matrix (Shape (10, 4)) ---
[[ 0.01823593  0.00966408 -0.07879221  0.0508922 ]
 [ 0.00921347 -0.09078637  0.05648828  0.02508462]
 [ 0.01735804 -0.05224469  0.01746349  0.01742316]]


In [89]:
def gradient(x):
    """x is the gradient from the next layer
    retruns the gradients of the current layer
    A VECTORIZED IMPLEMENATION FOR SOFTMAX GRADIENTS
    """
    M, C = x.shape # number of examples, number of classes
    diag_matrix = (x.reshape(M, C, 1) * np.eye(C))
    outer_mult = (x.reshape(M, 1, C) @ x.reshape(M, C, 1))
    J = diag_matrix - outer_mult
    return J
def backward(cache, grad_output):
    J = gradient(cache)
    M, C, _ = J.shape
    return (grad_output.reshape(M, C, 1) @ J).squeeze(-1)


In [76]:
(P.reshape(10, 1, 4) @ ((P.reshape(10, 1, 4) * np.eye(4)) - (P.reshape(10, 1, 4) @ P.reshape(10, 4, 1)))).squeeze()

array([[-0.31331809, -0.33723358, -0.301596  , -0.08757139],
       [-0.39050547, -0.39050519, -0.07990173, -0.33607045],
       [-0.28891095, -0.09098416, -0.28854375, -0.28868445],
       [-0.08038469, -0.42048138, -0.41836377, -0.35610023],
       [-0.37038081, -0.06251469, -0.36494715, -0.38320054],
       [-0.04436242, -0.48133312, -0.46908024, -0.49529389],
       [-0.28807808, -0.21235024, -0.30772729, -0.13995857],
       [-0.61439713, -0.61058852, -0.61440663, -0.0171625 ],
       [-0.35338446, -0.40930516, -0.07744079, -0.41150639],
       [-0.19282963, -0.37659824, -0.3704244 , -0.19724616]])

Mathematical Equivalence Match: True
Difference (Max Absolute Error): 1.0245693182753257e-12

--- Performance (M=10, C=3) ---
Generalized Approach: 0.00169s
Specialized Approach: 0.00074s
Speedup: 2.3x


In [86]:
grad_L_wrt_P_large

array([[-0.        , -0.        , -0.12628734],
       [-0.        , -0.48383658, -0.        ],
       [-0.        , -0.28469443, -0.        ],
       [-0.2882289 , -0.        , -0.        ],
       [-0.        , -0.        , -0.70951207],
       [-0.        , -0.        , -0.67888091],
       [-0.        , -0.        , -0.2588631 ],
       [-0.1952421 , -0.        , -0.        ],
       [-0.35236601, -0.        , -0.        ],
       [-0.        , -0.10617854, -0.        ]])

In [88]:
specialized_backward(P_large, Y_large)

array([[ 0.01331638,  0.00749912, -0.0208155 ],
       [ 0.04639212, -0.07933186,  0.03293975],
       [ 0.00991395, -0.06487462,  0.05496067],
       [-0.06530535,  0.03492383,  0.03038153],
       [ 0.05176686,  0.03413894, -0.08590581],
       [ 0.06668133,  0.01858855, -0.08526988],
       [ 0.0545415 ,  0.00682804, -0.06136954],
       [-0.04878154,  0.01348676,  0.03529477],
       [-0.07162042,  0.02376709,  0.04785333],
       [ 0.00453173, -0.00581901,  0.00128728]])

In [90]:
backward(P_large, grad_L_wrt_P_large)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)